In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler, LabelEncoder, MinMaxScaler,  Imputer
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR

from tqdm import tqdm
import time
# import cPickle as pickle
from scipy import sparse
from scipy.sparse import hstack

%matplotlib inline
plt.rcParams["figure.figsize"] = (15, 8)
pd.options.display.float_format = '{:.2f}'.format

In [2]:
df = pd.read_csv('data/vk_users_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34011 entries, 0 to 34010
Data columns (total 22 columns):
bdate         27938 non-null object
langs         4425 non-null object
political     5193 non-null float64
smoking       5987 non-null float64
verified      34011 non-null int64
life_main     5965 non-null float64
alcohol       5915 non-null float64
tv            3272 non-null object
country       32669 non-null object
sex           34011 non-null int64
last_name     34011 non-null object
age           34011 non-null float64
id            34011 non-null int64
posts         34011 non-null object
interests     5383 non-null object
university    14180 non-null float64
first_name    34011 non-null object
city          31156 non-null object
religion      6096 non-null object
movies        4427 non-null object
relation      14180 non-null float64
music         4709 non-null object
dtypes: float64(7), int64(3), object(12)
memory usage: 5.7+ MB


In [3]:
df['interests'] = df['interests'].fillna("empty")
df['movies'] = df['movies'].fillna("empty")
df['music'] = df['music'].fillna("empty")
df['tv'] = df['tv'].fillna("empty")
# заменим не указанные значения категориальных признаков на -1
df = df.fillna(-1)
# В столбце university содержится id университета по БД VK. 
# Будем считать, что если он указан, то у пользователя есть высшее 
# образование, иначе нет (хотя строго это не так, он может быть просто 
# не указан. Но примем такое предположение)
df['high_education'] = df['university'].apply(lambda x: 0 if x < -0.5 else 1)
df['age'] = df['age'].astype(int)
df['political'] = df['political'].astype(int)
df['smoking'] = df['smoking'].astype(int)
df['alcohol'] = df['alcohol'].astype(int)
df['relation'] = df['relation'].astype(int)
df['life_main'] = df['life_main'].astype(int)
# df['posts'] = df['posts'].str.decode('utf-8')

In [4]:
# Чтобы превратить нашу задачу в задачу классификации, уменьшим количество категорий возраста
# рассмотрим возраста от 0 до 18; от 18 до 30; от 30 до 50; от 50 до 70 и от 70 до 110 - 5 категорий
def age_cat(age):
    if 0 <= age <= 18:
        return 0
    elif 18 < age <= 30:
        return 1
    elif 30 < age <= 50:
        return 2
    elif 50 < age <= 70:
        return 3
    elif 70 < age <= 110:
        return 4
    
df['age_category'] = df['age'].apply(lambda x: age_cat(x))
df.head()

,bdate,langs,political,smoking,verified,life_main,alcohol,tv,country,sex,...,interests,university,first_name,city,religion,movies,relation,music,high_education,age_category
0,28.4,-1,-1,-1,0,-1,-1,empty,Россия,2,...,empty,-1.00,Виталя,Москва,-1,empty,-1,empty,0,4
1,-1,-1,-1,-1,0,-1,-1,empty,Россия,2,...,"🔞Не лайкаю, видео не смотрю, в группы не вступ...",0.00,Олег,-1,-1,empty,0,☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝☝,1,2
2,-1,-1,-1,-1,1,-1,-1,empty,Россия,2,...,empty,0.00,Артур,Набережные Челны,4ayan.ru,empty,0,empty,1,2
3,1.1,-1,8,-1,0,-1,-1,"Zara, Pull and Bear, Massimo Dutti, Bershka, S...",Украина,2,...,"Zara, Pull and Bear, Massimo Dutti, Bershka, S...",1892.00,Zara,Днепропетровск (Днепр),Buisness,"Zara, Pull and Bear, Massimo Dutti, Bershka, S...",0,"Zara, Pull and Bear, Massimo Dutti, Bershka, S...",1,2
4,6.6.1989,-1,-1,-1,0,-1,-1,empty,Украина,1,...,empty,-1.00,Наталя,Львов,-1,empty,-1,empty,0,1


In [5]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [6]:
df_train.head()

,bdate,langs,political,smoking,verified,life_main,alcohol,tv,country,sex,...,interests,university,first_name,city,religion,movies,relation,music,high_education,age_category
11153,25.3.1936,-1,-1,-1,0,-1,-1,empty,Россия,2,...,empty,-1.00,Михаил,Нижний Новгород,-1,empty,-1,empty,0,4
6352,12.8,"['Русский', 'English']",-1,2,0,-1,1,empty,Россия,2,...,empty,0.00,Валера,Самара,Пастафарианство,empty,2,empty,1,0
27363,1.9.2001,-1,-1,-1,0,-1,-1,empty,Россия,2,...,empty,0.00,Никита,Краснодар,-1,empty,6,empty,1,0
19244,29.8.1991,"['Русский', 'Українська']",3,4,0,2,5,Шоу Опры,Россия,1,...,"Клубы, парни, танцы",327.00,Марина,Москва,Православие,"Универ, Интерны, Один дома, Крик, Звонок",5,"Та, под которую можно подвигаться",1,1
29708,29.11.1945,-1,-1,-1,0,-1,-1,empty,Россия,2,...,empty,-1.00,Гриша,Радужный,-1,empty,-1,empty,0,4


In [7]:
df_test.head()

,bdate,langs,political,smoking,verified,life_main,alcohol,tv,country,sex,...,interests,university,first_name,city,religion,movies,relation,music,high_education,age_category
11313,27.8,-1,3,1,0,1,1,empty,Россия,1,...,empty,0.00,Анастасия,Краснодар,Православие,empty,4,empty,1,4
23533,19.3,"['Русский', 'Gagauz dili', 'Türkçe', 'Українсь...",3,2,0,1,2,не на вижу,Украина,2,...,"кушать, спать, интернет)",0.00,Dima,Одесса,Православие,разные,4,разные,1,1
3386,-1,-1,-1,4,0,3,4,empty,Россия,2,...,empty,0.00,Саша,-1,-1,empty,0,empty,1,4
12659,20.6,['English'],-1,2,0,-1,2,empty,Беларусь,2,...,empty,0.00,Алексей,Слуцк,-1,empty,0,"SUICIDEBOY, BONES, LIL PEEP, IMAGINE DRAGONS, ...",1,4
8037,31.3.1905,-1,-1,-1,0,-1,-1,empty,Россия,2,...,empty,-1.00,Денис,-1,-1,empty,-1,empty,0,4


In [8]:
# Сохраним наборы данных для дальнейшего использования
df_train.to_csv('learning/df_train.csv', sep='\t', encoding='utf-8')
df_test.to_csv('learning/df_test.csv', sep='\t', encoding='utf-8')

Смысл значений в столбцах sex, political, smoking, alcohol, relation, life_main следующий (из описания API VK):

1) sex - пол пользователя:
1 - женский;
2 - мужской;
0 - пол не указан

2) political - политические предпочтения:

1 - коммунистические;
2 - социалистические;
3 - умеренные;
4 - либеральные;
5 - консервативные;
6 - монархические;
7 - ультраконсервативные;
8 - индиффирентные;
9 - либертарианские;

3) smoking, alcohol - отношение к курению, алкоголю:

1 - резко негативное;
2 - негативное;
3 - компромиссное;
4 - нейтральное;
5 - положительное;

4) relation - семейное положение:

1 - не женат/не замужем;
2 - есть друг/есть подруга;
3 - помолвлен/помолвлена;
4 - женат/замужем;
5 - всё сложно;
6 - в активном поиске;
7 - влюблён/влюблена;
8 - в гражданском браке;
0 - не указано;

5) life_main - главное в жизни:

1 - семья и дети;
2 - карьера и деньги;
3 - развлечения и отдых;
4 - наука и исследования;
5 - совершенствование мира;
6 - саморазвитие;
7 - красота и искусство;
8 - слава и влияние

Попытаемся предсказать возраст (по данным VK (скорее всего, может быть с ошибками: неизвестно, как он там определяется)) по постам пользователя, его полу (пол указан всегда) и указанным признаками из раздела personal, а также по наличию высшего образования

#### Функции преобразования постов пользователя

In [9]:
from nltk.corpus import stopwords

stopwords_en = stopwords.words('english')
stopwords_ru = stopwords.words('russian')
stopwords_ge = stopwords.words('german')

stopwords_all = stopwords_en + stopwords_ru + stopwords_ge

# дополнительные буквосочетания, которые будем удалять из текстов
additional_stopwords = \
[u'https', u'vk', u'com', u'id', u'ph', u'др', u'св', u'ff', u'la', u'это', \
 u'de', u'pa', u'bb', u'p', u'ул', u'ин', u'http', u'ru', u'md', u'x', \
 u'ft', u'сб', u'b', u'к', u'www', u'youtube', u'ка', u'v', u'g', u'goo', u'gl', \
 u'eu', u'u', u'te', u'un', u'вк', u'w', u'ly', u'su', u'bu', u'vl', u'эт', u'r', u'e', \
 u'свой', u'ещё', u'мой', u'весь', u'днём', u'youtu', u'твой', u'наш', u'ваш', u'тот', u'этот']

stopwords_all = stopwords_all + additional_stopwords

# дополнительные удаляемые после лемматизации слова
delete_words = [u'свой', u'ещё', u'мой', u'весь', u'днём', u'youtu', u'твой', u'наш', u'ваш', u'тот', u'этот']

In [10]:
# используем для лемматизации библиотеку pymorphy2, которая работает с русским языком
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatization(text):
    return morph.parse(text)[0].normal_form

def my_tokenizer(text):
    text = text.lower()
    # очистка от html-разметки
    text = re.sub('<[^>]*>', '', text)
    # выделение смайлов
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # удаление несловарных символов
    text = re.sub(r'[\W]+', ' ', text, flags=re.U) + ' '.join(emoticons).replace('-', '')
    # удалим также все цифры, которые вряд ли полезны при анализе текста
    text = re.sub(r'\d+', '', text, flags=re.U)
    # удалим два или более пробелов
    text = re.sub(r'[ ]{2,}', '', text, flags=re.U)
    # удалим подчёркивания
    text = re.sub(r'[_]+', '', text, flags=re.U)
    # разбиение по пробелам и удаление стоп-слов
    tokenized = [w for w in text.split() if w not in stopwords_all]
    tokenized = [re.sub(r"\W", "", w, flags=re.U) for w in tokenized]
    # лемматизация
    tokenized = [lemmatization(w) for w in tokenized]
    # удалим все слова из одной и двух букв. Вряд ли они несут большой смысл. 
    # Также удалим "слова-паразиты"
    tokenized = [w for w in tokenized if len(w) > 2 and w not in delete_words]
    return tokenized

In [11]:
def get_posts_col(df):
    return df['posts']

def get_sex_col(df):
    return df[['sex']]

def get_interests_col(df):
    return df[df['interests'] != ""]['interests']

def get_movies_col(df):
    return df[df['movies'] != ""]['movies']

def get_music_col(df):
    return df[df['music'] != ""]['music']

def get_tv_col(df):
    return df[df['tv'] != ""]['tv']

def get_categorial_cols(df):
    return df[['high_education', 'political', 'smoking', 'alcohol', 'relation', 'life_main']]

def get_text_sample(df):   
    return df[['posts','interests', 'movies', 'music', 'tv']]

vec = make_union(*[
    make_pipeline(FunctionTransformer(get_categorial_cols, validate=False), MinMaxScaler()),
    make_pipeline(FunctionTransformer(get_posts_col, validate=False)), CountVectorizer(tokenizer=my_tokenizer)
])

С pipeline'ом что-то не получается, поэтому используем только столбец текста

In [12]:
y_train = df_train['age_category']
y_test = df_test['age_category']
print(y_train.shape)
print(y_test.shape)

(27208,)
(6803,)


In [13]:
y_train.to_csv('learning/y_train', sep='\t')
y_test.to_csv('learning/y_test', sep='\t')

In [14]:
# функции для сохранения и последующего восстановления разреженной матрицы
def save_sparse_csr(filename, array):
    np.savez(filename, data=array.data, indices=array.indices,
             indptr=array.indptr, shape=array.shape)

def load_sparse_csr(filename):
    loader = np.load(filename)
    return csr_matrix((loader['data'], loader['indices'], loader['indptr']),
                      shape=loader['shape'])

In [15]:
# vectorizer = CountVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 2))
vectorizer_posts = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=1000)
vectorizer_interests = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=1000)
vectorizer_movies = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=1000)
vectorizer_music = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=1000)
vectorizer_tv = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(1, 1), analyzer='word', max_features=1000)

In [16]:
# Используем сразу все текстовые колонки
X_train_posts = vectorizer_posts.fit_transform(tqdm(get_posts_col(df_train)))
print(X_train_posts.shape)

X_train_interests = vectorizer_interests.fit_transform(tqdm(get_interests_col(df_train)))
print(X_train_interests.shape)

X_train_movies = vectorizer_movies.fit_transform(tqdm(get_movies_col(df_train)))
print(X_train_movies.shape)

X_train_music = vectorizer_music.fit_transform(tqdm(get_music_col(df_train)))
print(X_train_music.shape)

X_train_tv = vectorizer_tv.fit_transform(tqdm(get_tv_col(df_train)))
print(X_train_tv.shape)

X_train_text = hstack([X_train_posts, X_train_interests, X_train_movies, X_train_music, X_train_tv])
X_train_text.shape

  1%|          | 145/27208 [00:00<00:34, 778.36it/s]

(27208, 1000)


  1%|          | 303/27208 [00:00<00:09, 2945.75it/s]

(27208, 1000)


  2%|▏         | 414/27208 [00:00<00:06, 4092.73it/s]

...

  2%|▏         | 430/27208 [00:00<00:06, 4184.71it/s]

(27208, 1000)


100%|██████████| 27208/27208 [00:07<00:00, 3414.18it/s]

(27208, 1000)


(27208, 5000)

### Рассмотрим получившуюся матрицу tf-idf для постов

In [17]:
features_posts = vectorizer_posts.get_feature_names()
features_interests = vectorizer_interests.get_feature_names()
features_movies = vectorizer_movies.get_feature_names()
features_music = vectorizer_music.get_feature_names()
features_tv = vectorizer_tv.get_feature_names()

In [21]:
print(features_posts[:10])
print(features_interests[:10])
print(features_movies[:10])
print(features_music[:10])
print(features_tv[:10])

[u'adid', u'android', u'app', u'bitcoin', u'club', u'facebook', u'fotomagicsu', u'fotomimi', u'happy', u'instagram']
[u'abc', u'agel', u'aimania', u'akuna', u'alfa', u'alivemax', u'ambre', u'amway', u'apple', u'aquel']
[u'alliance', u'american', u'bad', u'big', u'black', u'capital', u'cash', u'club', u'corp', u'dead']
[u'abba', u'accept', u'acid', u'adele', u'adriano', u'aerosmith', u'age', u'air', u'aka', u'akon']
[u'air', u'alliance', u'allin', u'american', u'androiid', u'apple', u'aston', u'bad', u'bbc', u'big']


In [22]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [23]:
def top_feats_in_doc(Xtr, features, row_id, top_n=25):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [25]:
top_feats_in_doc(X_train_posts, features_posts, 1000, top_n=6)

,feature,tfidf
0,мор,0.49
1,music,0.22
2,встреча,0.22
3,настроение,0.20
4,ёлка,0.20
5,дом,0.18


In [26]:
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)

In [27]:
top_mean_feats(X_train_posts, features_posts, top_n=10)

,feature,tfidf
0,человек,0.04
1,app,0.04
2,рождение,0.03
3,который,0.03
4,открытка,0.03
5,друг,0.03
6,год,0.03
7,жизнь,0.03
8,любить,0.03
9,новый,0.02


In [28]:
def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [29]:
top_feats_by_class(X_train_posts, y_train, features_posts, top_n=5)

[    feature  tfidf
 0       app   0.04
 1      друг   0.03
 2  рождение   0.03
 3    узнать   0.03
 4    любить   0.03,      feature  tfidf
 0    человек   0.04
 1    который   0.04
 2  instagram   0.04
 3    спасибо   0.03
 4       друг   0.03,     feature  tfidf
 0       app   0.05
 1  открытка   0.05
 2   человек   0.04
 3   который   0.03
 4  рождение   0.03,     feature  tfidf
 0       app   0.06
 1  открытка   0.06
 2   человек   0.04
 3  рождение   0.03
 4   который   0.03,     feature  tfidf
 0   человек   0.04
 1  рождение   0.04
 2    любить   0.03
 3     жизнь   0.03
 4   который   0.03]

In [32]:
save_sparse_csr('learning/X_train_bag_of_words', X_train_text)

AttributeError: indices not found

In [21]:
# сохраним векторизатор, чтобы не тратить время на повторное обучение
with open('learning/vectorizer.pk', 'wb') as fin:
    pickle.dump(vectorizer, fin)

In [13]:
# vectorizer = pickle.load(open('vectorizer.pk', 'rb'))
# vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=<function my_tokenizer at 0x7fdf1aef2e60>, use_idf=True,
        vocabulary=None)

In [36]:
# Функция для получения вектора признаков для обучения в виде разреженной матрицы
def get_X(X_bag_of_words, df):
    scaler = MinMaxScaler()

    # прибавим остальные признаки, которые будем учитывать при обучении модели
    X_sex = sparse.csr_matrix(get_sex_col(df))

    categorial_cols_scale = scaler.fit_transform(get_categorial_cols(df))
    X_categorial = sparse.csr_matrix(categorial_cols_scale)

    X = hstack([X_sex, X_categorial, X_bag_of_words])

    print(X.shape)
    
    return X

In [37]:
X_train = get_X(X_train_text, df_train)

(27208, 5007)


In [38]:
X_test_posts = vectorizer_posts.transform(tqdm(get_posts_col(df_test)))
X_test_interests = vectorizer_interests.transform(tqdm(get_interests_col(df_test)))
X_test_movies = vectorizer_movies.transform(tqdm(get_movies_col(df_test)))
X_test_music = vectorizer_music.transform(tqdm(get_music_col(df_test)))
X_test_tv = vectorizer_tv.transform(tqdm(get_tv_col(df_test)))

X_test_text = hstack([X_test_posts, X_test_interests, X_test_movies, X_test_music, X_test_tv])

100%|██████████| 6803/6803 [00:01<00:00, 3680.27it/s]


In [39]:
X_test = get_X(X_test_text, df_test)

(6803, 5007)


### Обучение модели

In [40]:
def randomized_cv(model, param_grid, x_train, y_train):
    grid_search = RandomizedSearchCV(model, param_grid, cv=5, scoring='accuracy', n_iter=10)
    t_start = time.time()
    grid_search.fit(x_train, y_train)
    t_end = time.time()
    print('model {} best accuracy score is {}'.format(model.__class__.__name__, grid_search.best_score_))
    print('time for training is {} seconds'.format(t_end - t_start))
    return grid_search.best_estimator_

In [41]:
param_grid = {'alpha':[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.5, 2, 5]}
model = MultinomialNB()
best_model = randomized_cv(model, param_grid, X_train, y_train)

model MultinomialNB best accuracy score is 0.460783593061
time for training is 3.3917350769 seconds


In [31]:
with open('learning/model_bayes.pk', 'wb') as fin:
    pickle.dump(best_model, fin)

NameError: name 'pickle' is not defined

In [42]:
y_pred = best_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.467146846979


Видим, что точность не слишком высокая, но всё же лучше случайного гадания (у нас 5 классов, поэтому при случайном выборе было бы 0.2). Есть ощущение, что текств постах ВК не слишком коррелирует с возрастом пользователя, поэтому хорошее предсказание сделать тяжело

In [43]:
# попробуем биграммы
vectorizer_2gram = TfidfVectorizer(tokenizer=my_tokenizer, ngram_range=(2, 2), analyzer='word', max_features=5000)
X_train_bag_of_words_2gram = vectorizer_2gram.fit_transform(tqdm(get_posts_col(df_train)))

100%|██████████| 27208/27208 [1:08:17<00:00,  6.64it/s]


In [45]:
X_train_2gram = get_X(X_train_bag_of_words_2gram, df_train)

(27208, 5007)


In [46]:
X_test_bag_of_words_2gram = vectorizer_2gram.transform(tqdm(get_posts_col(df_test)))

100%|██████████| 6803/6803 [17:17<00:00,  6.56it/s]


In [47]:
X_test_2gram = get_X(X_test_bag_of_words_2gram, df_test)

(6803, 5007)


In [48]:
model_2gram = MultinomialNB()
best_model_2gram = randomized_cv(model_2gram, param_grid, X_train_2gram, y_train)

/usr/local/lib/python2.7/dist-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


model MultinomialNB best accuracy score is 0.475926198177
time for training is 1.65328288078 seconds


In [49]:
y_pred_2gram = best_model_2gram.predict(X_test_2gram)
print(accuracy_score(y_test, y_pred_2gram))

0.474496545642


In [60]:
# рассмотрим вариант без использования определённого нами tokenizer'а
vectorizer_standard = TfidfVectorizer(max_features=10000)
X_train_bag_of_words_standard = vectorizer_standard.fit_transform(tqdm(get_posts_col(df_train)))

100%|██████████| 27208/27208 [00:22<00:00, 1183.00it/s]


In [61]:
X_train_standard = get_X(X_train_bag_of_words_standard, df_train)

(27208, 10007)


In [62]:
X_test_bag_of_words_standard = vectorizer_standard.transform(tqdm(get_posts_col(df_test)))

100%|██████████| 6803/6803 [00:06<00:00, 1046.56it/s]


In [63]:
model_standard = MultinomialNB()
best_model_standard = randomized_cv(model_standard, param_grid, X_train_standard, y_train)

model MultinomialNB best accuracy score is 0.488496030579
time for training is 9.50514888763 seconds


In [64]:
X_test_standard = get_X(X_test_bag_of_words_standard, df_test)
y_pred_standard = best_model_standard.predict(X_test_standard)
print(accuracy_score(y_test, y_pred_standard))

(6803, 10007)
0.492870792298


### Некоторые выводы

1) Текстовые данные VK и возраст пользователя коррелирую, но незначительно. Используя только тестовые данные, предсказать возраст с высокой точностью не получится. Достигнутая точность составляет около 0,5 на 5 классах, что примерно в 2,5 раза лучше случайного гадания. 

2) Использование биграмм (пар слов) несколько увеличивает точность, но незначительно.

3) Выбор количества "оставляемых" в наборе слов влияет на точность, но как выбрать оптималдьное значение, не слишком понятно. 